In [ ]:
import pandas as pd
import numpy as np
from trade_tester.tester import TesterBaseClass, render_candles
from trade_tester.data_classes import Actions, Order
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler

In [ ]:
klines_file = 'klines/DOGEUSDT_15m'

In [ ]:
klines = pd.read_csv(f'{klines_file}.csv')[['open_time', 'open', 'close', 'high', 'low', 'vol', 'trades']]
klines = klines.rename({'open_time': 'date'}, axis=1)
klines['date'] = pd.to_datetime(klines['date'], unit='ms')
# klines = klines.iloc[-500:].reset_index(drop=True)

In [ ]:
class TesterClass(TesterBaseClass):

    def _on_tick(self, action: Actions) -> Order:
        open_orders = self.open_orders()

        # close open orders
        tick_pnl = 0
        for order in open_orders:
            if order.type == Actions.Buy:
                # Buy
                if self.tick['open'] >= order.TP or self.tick['open'] <= order.SL:
                    tick_pnl += self.close_order(order)
                    # self.done = True
            else:
                # Sell
                if self.tick['open'] <= order.TP or self.tick['open'] >= order.SL:
                    tick_pnl += self.close_order(order)
                    # self.done = True

        # Open orders.
        if not self.done and not open_orders:
            _tp = self.tick['open'] * self.TP/100
            _sl = self.tick['open'] * self.SL/100
            if action == Actions.Buy:
                _tp = self.tick['open'] + _tp
                _sl = self.tick['open'] - _sl
            else:
                _tp = self.tick['open'] - _tp
                _sl = self.tick['open'] + _sl

            self.open_order(
                action=Actions(action),
                vol=self.balance,
                TP=_tp,
                SL=_sl,
            )
        
        # Если свеча случайно оказалась прибыльной, а за ней все плохо, то обнуляем прибыль
        for order in self.orders:
            if order.close:
                if order.open > self.tick['open']:
                    order.pnl = -20
                    order.pnl_percent = -2
                    self.done = True
        
        # Trailing order
        # if self.n_tick >= 3:
        #     for order in open_orders:
        #         sl = klines.at[self.n_tick-3, 'open']
        #         if sl > order.SL:
        #             order.SL = sl 

        # return the only one order
        return self.orders[0]
        

In [ ]:
# тейк от 0.1 до 5%
levels = np.arange(0.5, 5, 0.1)
klines['buy_prob'] = 0
best_orders = []
for i in range(len(klines)):
    # удерживаем позиция максимум 15 свечей
    df = klines.iloc[i: i+15]
    order_list = []
    for level in levels:
        tester = TesterClass(klines=df, TP=level, SL=level/2)
        while not tester.done:
            order_list.append(tester.on_tick(action=Actions.Buy))

    max_profit = float('-inf')
    index_max = 0
    for n, order in enumerate(order_list):
        if max_profit < order.pnl:
            max_profit = order.pnl
            index_max = n

    if max_profit > 0:
        if order_list[index_max].close:
            best_orders.append(order_list[index_max])
    klines.at[i, 'buy_prob'] = max_profit
    if i % 200 == 0 and i != 0:
        print(f'kline {i} in {len(klines)}')
print('done!')
klines['buy_prob'] = klines['buy_prob'].apply(lambda x: 0 if x < 0 else 1)
klines = klines[:-15]
klines.to_csv(f'{klines_file}_prob2.csv', index=False)

In [ ]:
df = pd.read_csv(f'{klines_file}_prob2.csv')
# df['buy_prob'] = df['buy_prob'].apply(lambda x: 40 if x > 40 else -40 if x < -40 else x)
df['trades'] = df['trades'].apply(lambda x: 100000 if x > 100000 else x)
df['vol'] = df['vol'].apply(lambda x: 2e8 if x > 2e8 else x)

In [ ]:
df['ksize'] = df['high'] - df['low']

In [ ]:
df

In [ ]:
import pickle

def scale_robust(df, column_name):
    scaler1 = RobustScaler()
    scaler2 = MinMaxScaler(feature_range=(0, 1))
    values = df[column_name].values.reshape(-1, 1)
    scaled_values = scaler1.fit_transform(values)
    scaled_values = scaler2.fit_transform(scaled_values)
    df_scaled = df.copy()
    df_scaled[column_name] = scaled_values
    # Сохранение scaler
    with open('scalers/robust.pkl', 'wb') as file:
        pickle.dump(scaler1, file)
    with open('scalers/minmax.pkl', 'wb') as file:
        pickle.dump(scaler2, file)
    return df_scaled
    
def scale_columns_std(df, cols):
    data = df[cols].values.astype(float)
    scaler = StandardScaler()
    scaler.fit(data.reshape(-1, 1))
    data_scaled = scaler.transform(data.reshape(-1, 1)).reshape(-1, len(cols))
    df_scaled = pd.DataFrame(data_scaled, index=df.index, columns=cols)
    df_remaining = df.drop(cols, axis=1)
    with open('scalers/standard.pkl', 'wb') as file:
        pickle.dump(scaler, file)
    return pd.concat([df_remaining, df_scaled], axis=1)

# df_scaled = scale_robust(df, 'buy_prob')
df_scaled = scale_robust(df, 'vol')
df_scaled = scale_robust(df_scaled, 'trades')
# df_scaled = scale_columns_std(df_scaled, ['open', 'close', 'high', 'low'])
df_scaled = df_scaled.reindex(columns=['date', 'open', 'close', 'high', 'low', 'vol', 'trades', 'ksize', 'buy_prob'])
df_scaled.to_csv(f'{klines_file}_prob_scaled.csv', index=False)

In [ ]:
df2 = df[-1000:]
# df = klines
fig = render_candles(df2, colored_deal_probability=True, prob_col='buy_prob')
fig.show()

In [ ]:
df2 = df_scaled.iloc[len(df)-1000:]
fig = render_candles(df2, colored_deal_probability=True, prob_col='buy_prob')
fig.show()